# Семинар 7: Actor-Critic


## Майнор ВШЭ, 7.03.2019

На этом семинаре будем рассматривать простой эксперимент с алгоритмом Actor-Critic.

![Модель Actor-Critic](https://cdn-images-1.medium.com/max/1600/1*-GfRVLWhcuSYhG25rN0IbA.png)


Модель актор-критик имеет два отдельных аппроксиматора: первый используется для предсказания того, какое действие предпринять, учитывая текущее состояние среды, и второй - чтобы найти значение полезности действия/состояния.

### Задание 1. Модель актора

Начнем с импортирования всех необходимых библиотек.

In [ ]:
import gym
import numpy as np 
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import Add, Multiply
from keras.optimizers import Adam
import keras.backend as K

import tensorflow as tf

import random
from collections import deque

Тестировать нашу модель будем на простой задаче с перевернутым маятником.

In [ ]:
## Создаем окружение Pendulum - смотрим как оно выглядит
# env = 

Начнем с определения модели актера. Цель модели состоит в том, чтобы, учитывая текущее состояние среды, определить наилучшее действие. Для начала будем использовать несколько полносвязных слоев, отображающих наблюдение среды в точку в пространстве среды.

In [ ]:
def create_actor_model(env):
    ### Задаем простую модель - 3-4 полносвязных слоя
    # state_input = 
    # output = 

    model = Model(input=state_input, output=output)
    adam  = Adam(lr=0.001)
    model.compile(loss="mse", optimizer=adam)
    return state_input, model

Далее мы хотим определить, какое изменение параметров (в модели актора) приведет к наибольшему увеличению значения Q (предсказанному моделью критика).

In [ ]:
learning_rate = 0.001

memory = deque(maxlen=2000)
actor_state_input, actor_model = create_actor_model(env)
_, target_actor_model = create_actor_model(env)

# where we will feed de/dC (from critic)
actor_critic_grad = tf.placeholder(tf.float32, [None, env.action_space.shape[0]]) 

actor_model_weights = actor_model.trainable_weights
# dC/dA (from actor)
# Вычисялем градиент и передаем его функции оптимизации
# actor_grads = 
# grads = 
# optimize = 

### Задание 2. Модель критика

Критик нужен для того, чтобы по состоянию окружающей среды и совершенному действию в качестве входных данных рассчитать соответствующую оценку. Построим его модель с помощью нескольких полвносвязных слоев с одним объединением перед окончательным предсказанием Q-значения.

In [ ]:
def create_critic_model(env):
    ### Задаем простую модель - 2 полносвязных слоя для состояний и 1 слой для действий
    # state_input = 

    # action_input = 

    ### Объдиняем оба выхода и далее добавляем еще пару полносвязных слоев
    # merged    =
    #output = 
    
    ### Все собираем в кучу
    # model  = 

    adam  = Adam(lr=0.001)
    model.compile(loss="mse", optimizer=adam)
    return state_input, action_input, model

Нам нужны ссылки как на входы действий, так и на входы по состоянию, поскольку нам нужно использовать их при обновлении для сети актора.

In [ ]:
critic_state_input, critic_action_input, critic_model = create_critic_model(env)
_, _, target_critic_model = create_critic_model(env)

# Вычисялем градиент критика
# where we calcaulte de/dC for feeding above
# critic_grads = 

In [ ]:
sess = tf.Session()
K.set_session(sess)
# Initialize for later gradient calculations
sess.run(tf.initialize_all_variables())

### Задание 3. Определяем метод обучения модели

Теперь определим метод обучения по памяти. Мы просто находим дисконтированное будущее вознаграждение и обучаемся на этом. Обучение проводим на паре состояние/действие и используем target_critic_model для прогнозирования будущей награды. Для актора мы уже определили как градиенты будут работать в сети, и теперь просто должны вызвать их с действиями и состояниями, которые берем из памяти.

In [ ]:
gamma = .95

def train(sess, memory):
    batch_size = 32
    if len(memory) < batch_size:
        return
    rewards = []
    ### Выбираем случай батч
    # samples = 
  
    ### Обучение модели актора
    for sample in samples:
        cur_state, action, reward, new_state, _ = sample
        ### Предсказываем действие
        # predicted_action = 
        ### Вычисляем градиент в сессии
        # grads = sess.run(...)[0]

        ### Делаем шаг оптимизациив сессии
        # sess.run(....)
        
    ### Обучение модели критика
    for sample in samples:
        cur_state, action, reward, new_state, done = sample
        if not done:
            ### Предсказываем действие
            # target_action = 
            ### Предсказываем будущее вознаграждение
            # future_reward = 
            ### Считаем будущее вознаграждение
            # reward ...
        ### Обучаем модель критика
        #critic_model...

В этой модели мы используем целевую сеть для улучшения сходимости. Мы должны обновлять ее веса на каждом временном шаге. Однако мы делаем это с меньшей частотой ,которая задается параметром $\tau$. Проделываем эту операцию как для актера так и для критика.

In [ ]:
def update_target():
    actor_model_weights  = actor_model.get_weights()
    actor_target_weights = target_critic_model.get_weights()

    for i in range(len(actor_target_weights)):
        actor_target_weights[i] = actor_model_weights[i]
    target_critic_model.set_weights(actor_target_weights)


    critic_model_weights  = critic_model.get_weights()
    critic_target_weights = critic_target_model.get_weights()

    for i in range(len(critic_target_weights)):
        critic_target_weights[i] = critic_model_weights[i]
    critic_target_model.set_weights(critic_target_weights)

### Задание 4. Проводим эксперименты.

Записываем полный цикл обучения: запоминиаем статистику и проводим на ней обучение.

In [ ]:
num_trials = 10000
trial_len  = 500
epsilon = 1.0
epsilon_decay = .995

cur_state = env.reset()
action = env.action_space.sample()
while True:
    env.render()
    cur_state = cur_state.reshape((1, env.observation_space.shape[0]))
    
    epsilon *= epsilon_decay
    ### Реализуем epsilon-жадную стратегию
    # action = 
    
    action = action.reshape((1, env.action_space.shape[0]))

    new_state, reward, done, _ = env.step(action)
    new_state = new_state.reshape((1, env.observation_space.shape[0]))

    ### Сохраняем в память данные
    # memory...
    train(sess, memory)

    cur_state = new_state

Постройте график вознаграждения в зависимости от номера эпизода.